In [1]:
from fastai import *
from fastai.tabular import *
import pandas as pd
from torchsummary import summary
import torch
from torch import nn
import imageio
import torch
import glob
from fastai.vision import *
import os
from torch import nn
import torch.nn.functional as F

In [4]:
colab = False

In [6]:
if colab:
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    %cd "/content/drive/My Drive/automatic-asset-classification"
    %ls "/content/drive/My Drive/automatic-asset-classification"
    image_path = "/content/drive/My Drive/automatic-asset-classification/data/final_dataset"
    
if not colab:
    image_path = '/Users/henriwoodcock/Documents/Code/data_projects/automatic-asset-classification/data/'\
                 'final_dataset/final'

In [7]:
size = 224
batchsize = 32
tfms = get_transforms(do_flip = False)
src = (ImageImageList.from_folder(image_path).split_by_rand_pct(seed=2).label_from_func(lambda x: x))
data = (src.transform(size=size, tfm_y=True)
        .databunch(bs=batchsize)
        .normalize(imagenet_stats, do_y = False))

In [8]:
class AutoEncoder(nn.Module):

    def __init__(self):
        super(AutoEncoder, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False);
        self.batch1 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True);
        
        self.up1 = nn.Upsample(scale_factor = 2, mode = 'bilinear');
        self.conv2 = nn.Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False);
        
        self.relu = nn.ReLU(inplace = True)
        
    def encoder(self, x):
        return nn.Sequential(self.conv1, self.batch1, self.relu)(x)
    
    def decoder(self, x):
        return nn.Sequential(self.up1,self.conv2, self.relu)(x)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [9]:
autoencoder = AutoEncoder();

In [10]:
learn = Learner(data, autoencoder, loss_func=F.mse_loss);

In [ ]:
learn.lr_find();

epoch,train_loss,valid_loss,time
0,0.246415,#na#,01:38


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/nn/functional.py:2494: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [5]:
data

ImageDataBunch;

Train: LabelList (387 items)
x: ImageImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
Path: /content/drive/My Drive/automatic-asset-classification/data/final_dataset;

Valid: LabelList (96 items)
x: ImageImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
Path: /content/drive/My Drive/automatic-asset-classification/data/final_dataset;

Test: None